# Applied Project in Big Data on Industrial Dataset

## DATA PROCESSING TECHNIQUES
## Part II. Full example how to collect data

### 1. Libraries

In [ ]:
import os
import re
import http
import json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from random import randint
from urllib.request import (
    Request, 
    urlopen)
from time import sleep
from bs4 import BeautifulSoup
from langdetect import detect
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

### 2. Some parameters and functions for scraping

In [ ]:
WORK_PATH = '.'
os.listdir(WORK_PATH + '/data')

In [ ]:
# example for sites from black-list
file_path = WORK_PATH + '/data/blacklist-HTTP-only.txt'
df = pd.read_csv(file_path, header=None)
print(df.shape)
display(df.head())

In [ ]:
# create url colunm to scrap sites
df.columns = ['url']
df['url'] = 'http://' + df['url']
display(df.head())

In [ ]:
# data folder
RAW_DATA_PATH = 'raw_data_black'

In [ ]:
# we need user agent to lokk like usual browser
USER_AGENT = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 YaBrowser/19.6.1.153 Yowser/2.5 Safari/537.36'
MIN_TIME_SLEEP = 1
MAX_TIME_SLEEP = 3
MAX_COUNTS = 2
TIMEOUT = 10

In [ ]:
def html_content(url_page, max_counts, timeout=10):
    """
    Function collects html page from site by url
      :max_count: maximum counts to retry get data from site
      :timeout: just keep it 10-15 seconds
    
    """
    counts = 0
    content = None
    while counts < max_counts:
        try:
            request = Request(url_page)
            request.add_header('User-Agent', USER_AGENT)
            response = urlopen(request, timeout=timeout)
            content = response.read().decode(response.headers.get_content_charset())
            break
        except Exception as e:
            counts += 1
            print('Error | ', url_page, ' | ', e, ' | counts: ', counts)
            sleep(randint(counts * MIN_TIME_SLEEP, counts * MAX_TIME_SLEEP))
    return content

### 3. Scraping loop

In [ ]:
# create data folder if not exists
os.makedirs(f'{WORK_PATH}/{RAW_DATA_PATH}', exist_ok=True)

In [ ]:
# loop for collecting sites
# some sites do not exist already, so errors could happen
# you can do the same loop for while-list files but save it to another folder

for url in tqdm(df.url):
    print('processing', url)
    html = html_content(url, max_counts=MAX_COUNTS, timeout=TIMEOUT)
    file_name = url.replace('http://', '')
    file_name = re.sub('[^a-zA-Z0-9_]', '', file_name)
    if html:
        with open(f'{WORK_PATH}/{RAW_DATA_PATH}/{file_name}.html', 'w', encoding='utf-8') as file:
            file.write(html)